In [86]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [87]:
df = pd.read_csv('fixedTesting/predictionsBoth.csv')
# df = df.drop(['taxa_obitos_covid',
#                 'taxa_atendimentos_dengue',
#                 'taxa_atendimentos_saude',
#                 'taxa_unidades_saude'], axis=1)
df


,Localidade,populacao_por_hectare,porcentagem_brancos,porcentagem_adultos,indiceEnvelhecimento,rendaMediacRendimento,rendaMedianacRendimento,porcentagem_cobertura,taxa_atendimentos_covid,preds
0,ALTO DA GLÓRIA,63.37,0.921593,0.750721,133.22,4101.11,3000.0,0.148813,0.416907,0.404430
1,SÃO MIGUEL,6.33,0.674838,0.681332,11.59,854.89,700.0,0.345552,0.172847,0.493791
2,BARREIRINHA,49.76,0.826442,0.723261,49.51,1600.64,1000.0,0.299122,0.369096,0.361857
3,ABRANCHES,30.56,0.774433,0.710668,32.04,1709.45,1000.0,0.375862,0.334445,0.399724
4,GUAÍRA,64.48,0.781200,0.716385,44.51,1696.24,1021.0,0.092372,0.387413,0.402622
...,...,...,...,...,...,...,...,...,...,...
70,CAMPINA DO SIQUEIRA,43.16,0.910456,0.740923,68.86,3083.02,1900.0,0.157678,0.292929,0.292929
71,SÃO FRANCISCO,45.26,0.889723,0.753670,134.11,2837.30,1800.0,0.159507,0.378630,0.378630
72,UMBARÁ,8.34,0.763641,0.699092,16.35,1164.04,850.0,0.412739,0.381367,0.381367
73,BOM RETIRO,27.16,0.904577,0.709077,121.89,2934.14,1800.0,0.224037,0.358805,0.358805


In [88]:
import ipywidgets as widgets
from IPython.display import display


gdf = gpd.read_file('DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp')
gdf = gdf.merge(df, left_on='NOME', right_on='Localidade', how='right')

gdf['error'] = gdf['taxa_atendimentos_covid'] - gdf['preds']
gdf['error_percentage'] = gdf['preds']/gdf['taxa_atendimentos_covid']


w = widgets.Dropdown(
    options=list(gdf),
    value='Localidade',
    description='Data:',
)

map_output = widgets.Output()

display(w, map_output)

Dropdown(description='Data:', index=10, options=('OBJECTID', 'CODIGO', 'TIPO', 'NOME', 'FONTE', 'CD_REGIONA', …

Output()

In [89]:
from IPython.display import clear_output

def update_map(change):
    with map_output:
        clear_output(wait=True)
        display(gdf.explore(column=change.new, legend=True, legend_kwds={"label": change.new, "orientation": "vertical"}))

w.observe(update_map, names='value')

# Initial map display
with map_output:
    display(gdf.explore(column=w.value, legend=True, legend_kwds={"label": w.value, "orientation": "vertical"}))

In [90]:
# Save the GeoDataFrame to a GeoJSON file
# gdf.to_file("curitiba_bairros_completo.geojson", driver='GeoJSON')


In [91]:
# # Create maps for each variable and save to file
# for column in gdf.columns:
#     if column != 'geometry':  # Skip the geometry column
#         fig, ax = plt.subplots(figsize=(12, 8))
#         gdf.plot(column=column, ax=ax, legend=True, cmap='viridis', edgecolor='grey', linewidth=0.2)
#         plt.title(f'Mapa de {column}')
#         plt.axis('off')
#         plt.tight_layout()
#         plt.savefig(f'maps/map_{column}.png', dpi=300, bbox_inches='tight')
#         plt.close(fig)  # Close the figure to free up memory

In [93]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


import numpy as np

# Define the custom color map
# cmap = mcolors.LinearSegmentedColormap.from_list("Diverging", ['bwr'])
divnorm=mcolors.TwoSlopeNorm(vmin=min(gdf['error']), vcenter=0., vmax=max(gdf['error']))
fig, ax = plt.subplots(figsize=(12, 8))
gdf.plot(column='error', ax=ax, legend=True, cmap='bwr', norm=divnorm, edgecolor='grey', linewidth=0.2)  # Added edgecolor and linewidth
plt.title(f'Mapa do erro absoluto')
plt.axis('off')
plt.tight_layout()
plt.savefig(f'map_error.png', dpi=300, bbox_inches='tight')
plt.close(fig)  # Close the figure to free up memory